In [1]:
import time
import sys
sys.path.append('../')
import librairies.dagfeaturingfx 
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,quantile_transform,PolynomialFeatures
from librairies.bt import *
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter
from librairies.strategy import *
scaler = MinMaxScaler(feature_range=(0, 1))
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait() 

Importing Librairies...


###### RELOAD LIBRAIRIE #####
from importlib import reload
sys.path.append('../')
librairies.dagfeaturingfx= reload(librairies.dagfeaturingfx)
from  librairies.dagfeaturingfx import *

%%time

_t1 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)

TICKER_LIST = ['EUR/AUD']
x=TICKER_LIST[0]
_ticker = x.replace('/','')
_period = 'm5'
df_all = pd.read_csv(x.replace('/','')+'_'+_period+'_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
df_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
df_all['Lindex'] = pd.to_datetime(df_all['Date'] + ' ' + df_all['Time'])
df_all.set_index(pd.to_datetime(df_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
df_all = df_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
df_all['WE'] = np.where(((df_all.index.weekday == 5) | (df_all.index.weekday == 6)),None,df_all.index.weekday)
df_all = df_all.dropna()
df_all = df_all.drop(['WE'],axis=1)

##### Calcul des averages pour les OHLC
df_all['Open'] = (df_all['OpenBid'] + df_all['OpenAsk']) / 2
df_all['High'] = (df_all['HighBid'] + df_all['HighAsk']) / 2
df_all['Low'] = (df_all['LowBid'] + df_all['LowAsk']) / 2
df_all['Close'] = (df_all['CloseBid'] + df_all['CloseAsk']) / 2

hourly_all = pd.read_csv(x.replace('/','')+'_H1_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
hourly_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
hourly_all['Lindex'] = pd.to_datetime(hourly_all['Date'] + ' ' + hourly_all['Time'])
hourly_all.set_index(pd.to_datetime(hourly_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
hourly_all = hourly_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
hourly_all['WE'] = np.where(((hourly_all.index.weekday == 5) | (hourly_all.index.weekday == 6)),None,hourly_all.index.weekday)
hourly_all = hourly_all.dropna()
hourly_all = hourly_all.drop(['WE'],axis=1)

##### Calcul des averages pour les OHLC
hourly_all['Open'] = (hourly_all['OpenBid'] + hourly_all['OpenAsk']) / 2
hourly_all['High'] = (hourly_all['HighBid'] + hourly_all['HighAsk']) / 2
hourly_all['Low'] = (hourly_all['LowBid'] + hourly_all['LowAsk']) / 2
hourly_all['Close'] = (hourly_all['CloseBid'] + hourly_all['CloseAsk']) / 2

engine.say("Processing featuring of dataframes, daily and intra-day")
engine.runAndWait()
##### Récupération des data pour tous les tickers sur la période demandée en intraday => df_all

engine.say("Raw data are loaded in memory")
engine.runAndWait()


df_all = timerange1D(df_all)
hourly_all = timerange1D(hourly_all)
daily_all = get_daily(hourly_all,TICKER_LIST)

engine.say("Daily up to date")
engine.runAndWait()

daily_all = timerange1W(daily_all)
weekly_all = get_weekly(daily_all,TICKER_LIST)
engine.say("Weekly up to date")
engine.runAndWait()

##### On calcule l'ADR sur le daily
daily_all = adr(daily_all,_window=14)
engine.say("ADR computed")
engine.runAndWait()

##### On récupère l'ADR qui a été calculé en daily (daily_all) pour le mettre dans la base intraday df_all
df_all = getadr(daily_all,df_all,TICKER_LIST)
engine.say("ADR get in daily")
engine.runAndWait()

df_all = adrhnl(daily_all,df_all,TICKER_LIST)

##### Calcul d'une SMA 200 sur df_all
df_all = sma(df_all=df_all,_window=200)

##### Calcul des bollinger sur df_all
df_all = bollinger(df_all,_slow=20)

##### Calcul du stochastic slow. Par défaut les paramètres sont 5 et 3.
df_all = slowstochastic(df_all,TICKER_LIST)

df_all = ema(df_all,21,TICKER_LIST)

df_all = ema(df_all,8,TICKER_LIST)

weekly_all = pivot(weekly_all,TICKER_LIST)

df_all = pivotimportdf(df_all,weekly_all,TICKER_LIST)

df_all = atr(df_all,TICKER_LIST,14)

df_all = rvi(df_all,TICKER_LIST,_window=14)

df_all = sbgamma(df_all,TICKER_LIST)

df_all = onhisma(df_all,TICKER_LIST,_window=5)
df_all = onlosma(df_all,TICKER_LIST,_window=5)

df_all = onhisma(df_all,TICKER_LIST,_window=21)
df_all = onlosma(df_all,TICKER_LIST,_window=21)

df_all = onhisma(df_all,TICKER_LIST,_window=34)
df_all = onlosma(df_all,TICKER_LIST,_window=34)

df_all = importohlc(df_all,weekly_all,TICKER_LIST,_suffix='_weekly')

df_all = importohlc(df_all=df_all,other_all=daily_all,TICKER_LIST=TICKER_LIST,_suffix='_daily')

engine.say("The job is done")
engine.runAndWait()


engine.say("AAll the bases are saved")
engine.runAndWait()

_t2 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)
print((_t2 - _t1))

%%time
joblib.dump(df_all,x.replace('/','')+'_DF_ALL')
joblib.dump(hourly_all,x.replace('/','')+'_HOURLY_ALL')
joblib.dump(weekly_all,x.replace('/','')+'_WEEKLY_ALL')
joblib.dump(daily_all,x.replace('/','')+'_DAILY_ALL')

In [2]:
%%time
x='EUR/AUD'
df_all = joblib.load(x.replace('/','')+'_DF_ALL')
hourly_all = joblib.load(x.replace('/','')+'_HOURLY_ALL')
weekly_all = joblib.load(x.replace('/','')+'_WEEKLY_ALL')
daily_all = joblib.load(x.replace('/','')+'_DAILY_ALL')


CPU times: user 255 ms, sys: 328 ms, total: 583 ms
Wall time: 596 ms


In [3]:
%%time
df_all = stochastic(df_all)

CPU times: user 55 ms, sys: 13.6 ms, total: 68.5 ms
Wall time: 48.9 ms


In [4]:
check_inf(df_all.drop(['Date','Symbol'],axis=1))


checking for infinity

printing the count of infinity values
It contains 4016 infinite values
It contains 4016 infinite values

printing column name where infinity is present
SB_Gamma    SB_Gamma
dtype: object

printing len(r),len(ds),len(df)
4016 807116 807116


In [5]:
check_nan(df_all.drop(['Date','Symbol'],axis=1))


checking for NaN

printing the count of NaN values
It contains 30478 NaN values
It contains 30478 NaN values

printing column name where NaN is present
ADR                  ADR
ADR_High        ADR_High
ADR_Low          ADR_Low
SMA_200          SMA_200
UpperBand      UpperBand
LowerBand      LowerBand
slow_K5          slow_K5
slow_D5          slow_D5
PP                    PP
S38                  S38
S62                  S62
S100                S100
S138                S138
S162                S162
S200                S200
R38                  R38
R62                  R62
R100                R100
R138                R138
R162                R162
R200                R200
ATR_14            ATR_14
RVI                  RVI
SB_Gamma        SB_Gamma
ONHISMA_5      ONHISMA_5
ONLOSMA_5      ONLOSMA_5
ONHISMA_21    ONHISMA_21
ONLOSMA_21    ONLOSMA_21
ONHISMA_34    ONHISMA_34
ONLOSMA_34    ONLOSMA_34
dtype: object

printing len(r),len(ds),len(df)
3966 807116 807116


In [6]:
features = featuring(df_all)

In [7]:
check_inf(features.drop(['Date','Symbol'],axis=1))


checking for infinity

printing the count of infinity values
It contains 4016 infinite values
It contains 4016 infinite values

printing column name where infinity is present
FSBGAMMA    FSBGAMMA
dtype: object

printing len(r),len(ds),len(df)
4016 807116 807116


In [8]:
check_nan(features.drop(['Date','Symbol'],axis=1))


checking for NaN

printing the count of NaN values
It contains 26545 NaN values
It contains 26545 NaN values

printing column name where NaN is present
FADRLo            FADRLo
FADRHi            FADRHi
FRVI40            FRVI40
FRVI60            FRVI60
FONLOSMA5      FONLOSMA5
FONHISMA5      FONHISMA5
FONLOSMA21    FONLOSMA21
FONHISMA21    FONHISMA21
FONLOSMA34    FONLOSMA34
FONHISMA34    FONHISMA34
FSBGAMMA        FSBGAMMA
FSMA200          FSMA200
FBOLUP20        FBOLUP20
FBOLLOW20      FBOLLOW20
FPP                  FPP
FS38                FS38
FS62                FS62
FS100              FS100
FS138              FS138
FS162              FS162
FS200              FS200
FR38                FR38
FR62                FR62
FR100              FR100
FR138              FR138
FR162              FR162
FR200              FR200
SBATR              SBATR
dtype: object

printing len(r),len(ds),len(df)
3966 807116 807116


In [14]:
%%time
_ticker = x.replace('/','')
hourly = df_all[df_all.Symbol==_ticker].copy()
_op1 = (hourly.Close - hourly.Open)/(hourly.Close.shift(1) - hourly.Open.shift(1))
_op2 = (hourly.Close - hourly.Open)/(hourly.CloseAsk.shift(1) - hourly.OpenAsk.shift(1))
_op3 = (hourly.Close - hourly.Open)/(hourly.CloseBid.shift(1) - hourly.OpenBid.shift(1))
_op4 = (hourly.Close - hourly.Open)/(hourly.CloseBid.shift(1) - hourly.OpenAsk.shift(1))
_op5 = (hourly.Close - hourly.Open)/(hourly.CloseAsk.shift(1) - hourly.OpenBid.shift(1))

_condition1 = hourly.Close.shift(1) != hourly.Open.shift(1)
_condition2 = hourly.CloseAsk.shift(1) != hourly.OpenAsk.shift(1)
_condition3 = hourly.CloseBid.shift(1) != hourly.OpenBid.shift(1)
_condition4 = hourly.CloseBid.shift(1) != hourly.OpenAsk.shift(1)
_condition5 = hourly.CloseAsk.shift(1) != hourly.OpenBid.shift(1)
    
hourly['SB_Gamma'] = np.where(_condition1,_op1,np.where(_condition2,_op2,np.where(_condition3,_op3,np.where(_condition4,_op4,np.where(_condition5,_op5,1.93E13)))))

CPU times: user 329 ms, sys: 87.1 ms, total: 416 ms
Wall time: 312 ms


In [15]:
check_inf(hourly.drop(['Date','Symbol'],axis=1))


checking for infinity

printing the count of infinity values
It contains 0 infinite values
It contains 0 infinite values

printing column name where infinity is present
Series([], dtype: object)

printing len(r),len(ds),len(df)
0 807116 807116


In [16]:
check_nan(hourly.drop(['Date','Symbol'],axis=1))


checking for NaN

printing the count of NaN values
It contains 30445 NaN values
It contains 30445 NaN values

printing column name where NaN is present
ADR                  ADR
ADR_High        ADR_High
ADR_Low          ADR_Low
SMA_200          SMA_200
UpperBand      UpperBand
LowerBand      LowerBand
slow_K5          slow_K5
slow_D5          slow_D5
PP                    PP
S38                  S38
S62                  S62
S100                S100
S138                S138
S162                S162
S200                S200
R38                  R38
R62                  R62
R100                R100
R138                R138
R162                R162
R200                R200
ATR_14            ATR_14
RVI                  RVI
SB_Gamma        SB_Gamma
ONHISMA_5      ONHISMA_5
ONLOSMA_5      ONLOSMA_5
ONHISMA_21    ONHISMA_21
ONLOSMA_21    ONLOSMA_21
ONHISMA_34    ONHISMA_34
ONLOSMA_34    ONLOSMA_34
dtype: object

printing len(r),len(ds),len(df)
3933 807116 807116


In [17]:
df_all[np.isinf(df_all.SB_Gamma)].SB_Gamma

Lindex
2010-01-04 15:45:00    inf
2010-01-07 17:05:00    inf
2010-01-07 20:10:00    inf
2010-01-11 06:55:00    inf
2010-01-11 09:50:00   -inf
                      ... 
2021-02-10 21:45:00    inf
2021-02-10 23:30:00   -inf
2021-02-11 18:55:00    inf
2021-02-12 14:15:00    inf
2021-02-15 08:45:00   -inf
Name: SB_Gamma, Length: 4016, dtype: float64

In [18]:
hourly[np.isinf(hourly.SB_Gamma)].SB_Gamma

Series([], Name: SB_Gamma, dtype: float64)

In [25]:
hourly.SB_Gamma.max()

19275406405145.0

In [35]:
1.93E13 - 19275406405145.0

4593594855.0

In [37]:
len(hourly[hourly.SB_Gamma==1.93E13])

0

In [24]:
df_all[(df_all.index>='2010-01-04 15:40:00') & (df_all.index<='2010-01-04 15:55:00')].SB_Gamma

Lindex
2010-01-04 15:40:00    0.000000
2010-01-04 15:45:00         inf
2010-01-04 15:50:00   -1.822785
2010-01-04 15:55:00    2.583333
Name: SB_Gamma, dtype: float64

In [48]:
hourly[(hourly.index>='2010-01-04 15:40:00') & (hourly.index<='2010-01-04 15:55:00')].SB_Gamma

Lindex
2010-01-04 15:40:00     0.000000
2010-01-04 15:45:00   -39.500000
2010-01-04 15:50:00    -1.822785
2010-01-04 15:55:00     2.583333
Name: SB_Gamma, dtype: float64

In [29]:
df_all[df_all.index=='2010-01-04 15:45:00'].Close[0],df_all[df_all.index=='2010-01-04 15:45:00'].Open[0]

(1.583, 1.582605)

In [30]:
df_all[df_all.index=='2010-01-04 15:40:00'].Close[0],df_all[df_all.index=='2010-01-04 15:40:00'].Open[0]

(1.582605, 1.582605)

In [28]:
df_all['Close'] = round(df_all.Close,6)

In [31]:
(df_all[df_all.index=='2010-01-04 15:45:00'].Close[0]-df_all[df_all.index=='2010-01-04 15:45:00'].Open[0])/ (df_all[df_all.index=='2010-01-04 15:40:00'].Close[0]-df_all[df_all.index=='2010-01-04 15:40:00'].Open[0])

inf

In [35]:
df_all[df_all.index=='2010-01-04 15:45:00']

,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Symbol,Open,...,ONLOSMA_34,Open_weekly,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal
Lindex,,,,,,,,,,,,,,,,,,,,,
2010-01-04 15:45:00,1.58233,1.58299,1.58205,1.58272,1.58288,1.58352,1.58267,1.58328,EURAUD,1.582605,...,1.586623,1.595525,1.59603,1.55618,1.560425,1.595525,1.59603,1.5771,1.580735,0


In [37]:
np.where(df_all.Open==df_all.Close,1,0).sum()

3523

In [38]:
np.where(df_all.OpenBid==df_all.CloseBid,1,0).sum()

10071

In [39]:
np.where(df_all.OpenAsk==df_all.CloseAsk,1,0).sum()

9977

In [40]:
np.where(df_all.OpenAsk==df_all.CloseBid,1,0).sum()

5780

In [41]:
np.where(df_all.OpenBid==df_all.CloseAsk,1,0).sum()

5936

In [51]:
test = pd.read_csv('AUDCAD.csv')

In [52]:
test

,Date,Timestamp,Open,High,Low,Close,Volume
0,20100101,12:00:00,0.94343,0.94343,0.94343,0.94343,1.669749
1,20100101,12:05:00,0.94353,0.94353,0.94340,0.94340,6.312595
2,20100101,12:10:00,0.94350,0.94350,0.94350,0.94350,1.975511
3,20100101,12:15:00,0.94343,0.94343,0.94343,0.94343,5.962901
4,20100101,12:20:00,0.94350,0.94350,0.94342,0.94342,6.765226
...,...,...,...,...,...,...,...
639411,20180720,08:05:00,0.97619,0.97631,0.97614,0.97624,416.189999
639412,20180720,08:10:00,0.97625,0.97685,0.97613,0.97669,618.719998
639413,20180720,08:15:00,0.97669,0.97680,0.97652,0.97674,366.219999
639414,20180720,08:20:00,0.97673,0.97679,0.97634,0.97634,477.110000


In [20]:
hourly[np.isnan(hourly.SB_Gamma)].SB_Gamma

Lindex
2010-01-04   NaN
Name: SB_Gamma, dtype: float64

In [43]:
check_nan(df_all.drop(['Date','Symbol'],axis=1))


checking for NaN

printing the count of NaN values
It contains 30478 NaN values
It contains 30478 NaN values

printing column name where NaN is present
ADR                  ADR
ADR_High        ADR_High
ADR_Low          ADR_Low
SMA_200          SMA_200
UpperBand      UpperBand
LowerBand      LowerBand
slow_K5          slow_K5
slow_D5          slow_D5
PP                    PP
S38                  S38
S62                  S62
S100                S100
S138                S138
S162                S162
S200                S200
R38                  R38
R62                  R62
R100                R100
R138                R138
R162                R162
R200                R200
ATR_14            ATR_14
RVI                  RVI
SB_Gamma        SB_Gamma
ONHISMA_5      ONHISMA_5
ONLOSMA_5      ONLOSMA_5
ONHISMA_21    ONHISMA_21
ONLOSMA_21    ONLOSMA_21
ONHISMA_34    ONHISMA_34
ONLOSMA_34    ONLOSMA_34
dtype: object

printing len(r),len(ds),len(df)
3966 807116 807116


In [91]:
df_all[(df_all.index=='2020-12-14 10:35:00')|(df_all.index=='2020-12-14 10:40:00')].iloc[:,:18]

,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Symbol,Open,High,Low,Close,Date,ADR,DayHigh,DayLow,ADR_High
Lindex,,,,,,,,,,,,,,,,,,
2020-12-14 10:35:00,1.60666,1.60699,1.60656,1.60666,1.60688,1.60719,1.60677,1.60688,EURAUD,1.60677,1.60709,1.606665,1.60677,2020-12-14,0.0108,1.61298,1.604535,1.615335
2020-12-14 10:40:00,1.60666,1.60679,1.60638,1.60667,1.60688,1.60701,1.60658,1.60687,EURAUD,1.60677,1.60690,1.606480,1.60677,2020-12-14,0.0108,1.61298,1.604535,1.615335


In [94]:
(df_all[(df_all.index=='2020-12-14 10:40:00')].Open[0] - df_all[(df_all.index=='2020-12-14 10:40:00')].Close[0]) / (df_all[(df_all.index=='2020-12-14 10:35:00')].Open[0] - df_all[(df_all.index=='2020-12-14 10:35:00')].Close[0])

nan

In [21]:
newfeatures = featuring(hourly)

In [22]:
len(df_all),len(features),len(hourly),len(newfeatures),len(df_all.dropna()),len(features.dropna()),len(hourly.dropna()),len(newfeatures.dropna())

(807116, 807116, 807116, 807116, 803150, 803150, 803183, 803183)

In [45]:
%%time
df_max = pd.DataFrame(index=df_all.T.index)
df_max = df_max.T
df_max

CPU times: user 28.1 s, sys: 348 ms, total: 28.5 s
Wall time: 28.5 s


,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Symbol,Open,...,ONLOSMA_34,Open_weekly,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal


In [54]:
%%time
def DagMaxBase(features_train):
    features_max = pd.DataFrame()
    for label in features_train.columns:
        features_max.loc[0,label] = features_train[label].max()
    joblib.dump(features_max,x.replace('/',''+'_MAX'))
    return(features_max)

NameError: name 'features_train' is not defined

In [50]:
features

,Symbol,Date,FEMA_21,FEMA_8,FADRLo,FADRHi,FRVI40,FRVI60,FONLOSMA5,FONHISMA5,...,FS162,FS200,FR38,FR62,FR100,FR138,FR162,FR200,SBATR,Signal
Lindex,,,,,,,,,,,,,,,,,,,,,
2010-01-04 00:00:00,EURAUD,2010-01-04,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2010-01-04 00:05:00,EURAUD,2010-01-04,-0.000177,-0.000152,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2010-01-04 00:10:00,EURAUD,2010-01-04,-0.001138,-0.000954,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2010-01-04 00:15:00,EURAUD,2010-01-04,-0.000439,-0.000233,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2010-01-04 00:20:00,EURAUD,2010-01-04,-0.000522,-0.000286,NaN,NaN,NaN,NaN,-0.000371,-0.000708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-15 23:35:00,EURAUD,2021-02-15,-0.000711,-0.000399,0.006002,-0.010082,24.201741,4.201741,-0.000147,-0.000214,...,0.009844,0.013868,-0.011226,-0.013712,-0.017737,-0.021761,-0.024247,-0.028272,-0.562484,0
2021-02-15 23:40:00,EURAUD,2021-02-15,-0.000746,-0.000396,0.005892,-0.009967,28.221246,8.221246,-0.000303,-0.000314,...,0.009734,0.013758,-0.011336,-0.013822,-0.017847,-0.021871,-0.024357,-0.028382,-0.295641,0
2021-02-15 23:45:00,EURAUD,2021-02-15,-0.000256,0.000054,0.006357,-0.009502,20.940659,0.940659,-0.000099,0.000227,...,0.010199,0.014223,-0.010871,-0.013357,-0.017382,-0.021406,-0.023892,-0.027917,1.182142,0


In [38]:
label

'CloseBid'

In [20]:
%%time
####### CHOIX DE LA TIME FRAME
_period = 'm5'
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = ['EUR/AUD']
TIK = ['JPY','CHF','CAD','GBP','AUD','NZD','SEK','NOK','MXN','ZAR','TRY','ILS','CNH','USD','HKD','0']
RATE = [105.4,0.8989,1.276,0.877,0.7676,0.7201,8.3684,8.5238,20.09,14.81,7.03,3.28,6.458,1,7.7527,1.21]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
#############################
##### F E A T U R I N G #####
#############################

print(df_all.shape,daily_all.shape,weekly_all.shape)
# Make our choice for the split, compliant to our backtest 
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2018-08-29' # stop the train and begin the test there '2016-08-31'
_stop = '2019-12-30' # stop the test there. After that, it is kept for oos
_last = '2021-02-15' # '2020-12-31'
#### S T R A T E G Y

df_all = stochastic(df_all)
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = _stop
_year_top = _last
_nb_bougie_exit = 2000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1/df_ratefx.loc[x[4:],'rate']
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target =  0.002
_exposure = 10
_size = 200000
_sl =  0.001
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only
print()
print(col.Fore.RED,'###############################################################################################')
print(' ####################################### OOS WITHOUT AI ########################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
##### Backtest Over Night
TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.CYAN,'###############################################################################################')
print(' #################################### TRAIN/TEST WITHOUT AI ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
_year_bottom = _start
_year_top = _stop

TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.BLUE,'###############################################################################################')
print(' #################################### DENOISING & ENHANCING ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
while _nb_looser > 0 :
    
    df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
    df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
    df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
    backtest = df_all[df_all.Symbol == x.replace('/','')]
    ##### Purification of signal by denoising and enhancing
    TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl)

df_all = joblib.load(x.replace('/','')+'_DF_ALL')
df_all = stochastic(df_all)
features = featuring(df_all)

features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# Let's isolate the ticker on which we made the test
features = features[features.Symbol==x.replace('/','')]

# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

# Proceed an MaxAbsScaler on features
features_train,features_test,features_oos = scaling(features,x.replace('/',''),TRACKER,_start,_mid,_stop,_last,scaler=MaxAbsScaler())
features_train,features_test,features_oos = quantile(features_train,features_test,features_oos,quantile_transform)

features_train = features_train.drop(['FSBGAMMA'],axis=1)
features_test = features_test.drop(['FSBGAMMA'],axis=1)
features_oos = features_oos.drop(['FSBGAMMA'],axis=1)

from sklearn.neural_network import MLPClassifier
_name = 'MLPClassifier'
_model = MLPClassifier(hidden_layer_sizes=(300,15), activation='relu', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=400, shuffle=True, random_state=26, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)

_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)


print('Signaux - Accuracy :' ,accu,'%')
print('Signaux - Precision :',prec,'%')
print('Signaux - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%')
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)

conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]

print(conf_matrix)
print()
print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
_tp = tt[0][0]
_fn = tt[0][1]
_prec = round((tt[0][0]/(tt[0][0]+tt[0][1]))*100,2)
_rec = round((tt[0][1]/(tt[0][0]+tt[0][1]))*100,2)

print('Vrais signaux trouvés    : ',tt[0][0])
print('Vrais signaux non trouvé :',tt[0][1])
print('Total des vrais signaux  :',tt[0][0]+tt[0][1])
if _prec > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif _prec < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Precision : ',_prec,'%')
if _rec > 69 and _rec > 69 :
    print(col.Fore.GREEN)
elif _rec < 51 or _rec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Recall',_rec,'%')
print(col.Style.RESET_ALL)

df_all_oos = df_all[(df_all.Date > _stop)&(df_all.Date <= _last)].dropna()

df_all_oos = df_all_oos[df_all_oos.Symbol==x.replace('/','')]


df_all_oos['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1))


df_all_oos['Signal'] = np.where((df_all_oos.Signal==1)&(df_all_oos.Valid==1),1,np.where((df_all_oos.Signal==-1)&(df_all_oos.Valid==1),-1,0))
 
##### On backtest selon le ticker selectionné sur la période déterminée

#joblib.dump(_model,'Save_'+x.replace('/','')+'_m5.sav')

print(x)
_year_bottom = _stop
_year_top = _last

#_trigger_invers = 1
#_target = 0.017
##### Backtest Over Night
FINAL_TRACKER = bt(df_all_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

(807116, 56) (2884, 7) (581, 19)
EUR/AUD

 ###############################################################################################
 ####################################### OOS WITHOUT AI ########################################
 ############################################################################################### 
Librairies imported

Début des opérations horodatée à 2021-03-07 15:53:25.120290

Chargement de la nouvelle base


 Le rate du ticker EUR/AUD est à  1.302761855132882 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 81786/81786 [00:04<00:00, 18319.44it/s]
 For ticker  EUR/AUD 
 
Total Profit & Loss : $  -138361.13 . En  1767   transactions.
 
Winners Number : 548 
 
Loosers number : 1219 
BT's execution time 0:00:10.816881
 EUR/AUD  results 
 Tested Period 2019-12-30  à 2021-02-15 
 Total Number of trades 1767 
Started Cash : 200000
P&L in currency:  -138361.13$ 
P&L in %:  -69.18% 
Average trade duration 30.68
# Winners  548.0